In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import sys,os,glob,json,pickle
from collections import OrderedDict

import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import utils
from enterprise.signals import gp_signals
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection

from enterprise_extensions import models,model_utils

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

%matplotlib inline

In [7]:
def get_noise_from_pal2(noisefile):
    psrname = noisefile.split('/')[-1].split('_noise.txt')[0]
    fin = open(noisefile, 'r')
    lines = fin.readlines()
    params = {}
    for line in lines:
        ln = line.split()
        if 'efac' in line:
            par = 'efac'
            flag = ln[0].split('efac-')[-1]
        elif 'equad' in line:
            par = 'log10_equad'
            flag = ln[0].split('equad-')[-1]
        elif 'jitter_q' in line:
            par = 'log10_ecorr'
            flag = ln[0].split('jitter_q-')[-1]
        elif 'RN-Amplitude' in line:
            par = 'red_noise_log10_A'
            flag = ''
        elif 'RN-spectral-index' in line:
            par = 'red_noise_gamma'
            flag = ''
        else:
            break
        if flag:
            name = [psrname, flag, par]
        else:
            name = [psrname, par]
        pname = '_'.join(name)
        params.update({pname: float(ln[1])})
    return params

In [8]:
#Where the enterprise 11yr datafiles are
topdir = os.getcwd()

runname = '/simGWB_3'
#Where the everything should be saved to (chains, cornerplts, histograms, etc.)
outdir = topdir + '/SimRuns' + runname

#The pickled pulsars
psr_pickle_file = outdir + '/enterprise_sim_pulsars.pickle'
noisepath = topdir + '/nano11/noisefiles_new/'
psrlistpath = topdir + '/nano11/psrlist_Tg3yr.txt'

In [10]:
#current_path = os.path.dirname(os.path.abspath(__file__))
current_path = os.getcwd()
print(current_path)
splt_path = current_path.split("/")
top_path_idx = splt_path.index('BackgroundInjections')
top_dir = "/".join(splt_path[0:top_path_idx+1])
print(top_dir)

/hyrule/data/users/akaiser/BackgroundInjections/SimInject1
/hyrule/data/users/akaiser/BackgroundInjections


In [5]:
encoding = "utf-8"
psrlist_bytes = np.loadtxt(psrlistpath,dtype='S42')
psrlist = []
for psr in psrlist_bytes:
    psrlist.append(psr.decode(encoding))

In [6]:
noisefiles = sorted(glob.glob(noisepath+'*.txt'))
noisefiles = [x for x in noisefiles if x.split('/')[-1].split('_')[0] in psrlist]

noise_params = {}
for nf in noisefiles:
    noise_params.update(get_noise_from_pal2(nf))

In [7]:
#Load pulsars from pickle file
with open(psr_pickle_file,'rb') as psrfile:
    psrs = pickle.load(psrfile)
    psrfile.close()

In [8]:
# find the maximum time span to set GW frequency sampling
selection = Selection(selections.by_backend)

tmin = [p.toas.min() for p in psrs]
tmax = [p.toas.max() for p in psrs]
Tspan = np.max(tmax) - np.min(tmin)

##### parameters and priors #####

# white noise parameters
'''
efac = parameter.Uniform(0.5,4.0)
log10_equad = parameter.Uniform(-10,-5)
log10_ecorr = parameter.Uniform(-10,-5)
'''
efac = parameter.Constant()
log10_equad = parameter.Constant()
log10_ecorr = parameter.Constant()

# red noise parameters
red_noise_log10_A = parameter.Uniform(-18,-13)
red_noise_gamma = parameter.Uniform(0,7)

# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw_1 = parameter.Uniform(-18,-13)('zlog10_A_gw')
gamma_gw_1 = parameter.Constant(13/3)('zgamma_gw')

# Second GW parameters
log10_A_gw_2 = parameter.Uniform(-18,-13)('zlog10_A_other_gw')
gamma_gw_2 = parameter.Constant(7/3)('zgamma_other_gw')

##### Set up signals #####

# timing model
tm = gp_signals.TimingModel()

# white noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
eq = white_signals.EquadNoise(log10_equad = log10_equad, selection=selection)
ec = white_signals.EcorrKernelNoise(log10_ecorr = log10_ecorr, selection=selection)

# red noise (powerlaw with 30 frequencies)
pl = utils.powerlaw(log10_A=red_noise_log10_A, gamma=red_noise_gamma)
rn = gp_signals.FourierBasisGP(spectrum=pl, components=30, Tspan=Tspan)

cpl_1 = utils.powerlaw(log10_A=log10_A_gw_1, gamma=gamma_gw_1)
cpl_2 = utils.powerlaw(log10_A=log10_A_gw_2, gamma=gamma_gw_2)

#Common red noise process with no correlations
crn_1 = gp_signals.FourierBasisGP(spectrum = cpl_1, components=30, Tspan=Tspan, name = 'gw')
crn_2 = gp_signals.FourierBasisGP(spectrum = cpl_2, components=30, Tspan=Tspan, name = 'other_gw')

# gwb with Hellings and Downs correlations
# Hellings and Downs ORF
#orf = utils.hd_orf()
#gwb_1 = gp_signals.FourierBasisCommonGP(cpl_1, orf, components=30, name='gw', Tspan=Tspan)
#gwb_2 = gp_signals.FourierBasisCommonGP(cpl_2, orf, components=30, name='other_gw', Tspan=Tspan)


# full model is sum of components
model = ef + eq + ec + rn + tm + crn_1 + crn_2  #+ crn

# initialize PTA
pta = signal_base.PTA([model(psr) for psr in psrs])

In [9]:
with open(outdir + '/parameters.json', 'w') as fp:
    json.dump(pta.param_names, fp)

In [10]:
#Set Default PTA parameters to the ones in the noisefiles
pta.set_default_params(noise_params)

INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_efac to 1.16587
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_efac to 1.11896
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_efac to 1.08538
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_efac to 1.38104
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_equad to -7.93502
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_equad to -6.17415
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_equad to -6.51038
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_PUPPI_log10_equad to -6.53715
INFO: enterprise.signals.signal_base: Setting B1855+09_430_ASP_log10_ecorr to -8.47348
INFO: enterprise.signals.signal_base: Setting B1855+09_430_PUPPI_log10_ecorr to -6.31096
INFO: enterprise.signals.signal_base: Setting B1855+09_L-wide_ASP_log10_ecorr to -6.09208
INFO: enterprise.signals.signal_base: Setting B185

INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr_800_GUPPI_log10_equad to -7.07038
INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr1_2_GUPPI_log10_ecorr to -7.95971
INFO: enterprise.signals.signal_base: Setting J0645+5158_Rcvr_800_GUPPI_log10_ecorr to -7.11304
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GASP_efac to 1.05488
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GUPPI_efac to 1.0814
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GASP_efac to 1.13901
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GUPPI_efac to 1.18258
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GASP_log10_equad to -6.46692
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr1_2_GUPPI_log10_equad to -6.50012
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GASP_log10_equad to -6.69741
INFO: enterprise.signals.signal_base: Setting J1012+5307_Rcvr_800_GUPPI_log10_equa

INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GUPPI_efac to 1.03918
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GASP_efac to 1.13277
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GUPPI_efac to 1.06578
INFO: enterprise.signals.signal_base: Setting J1713+0747_S-wide_ASP_efac to 1.11321
INFO: enterprise.signals.signal_base: Setting J1713+0747_S-wide_PUPPI_efac to 1.11321
INFO: enterprise.signals.signal_base: Setting J1713+0747_L-wide_ASP_log10_equad to -7.55522
INFO: enterprise.signals.signal_base: Setting J1713+0747_L-wide_PUPPI_log10_equad to -7.89216
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GASP_log10_equad to -7.28642
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr1_2_GUPPI_log10_equad to -7.99859
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GASP_log10_equad to -6.95725
INFO: enterprise.signals.signal_base: Setting J1713+0747_Rcvr_800_GUPPI_log10_equad to -7.1468
INF

INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_efac to 1.05106
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_efac to 0.988698
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_efac to 1.04206
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_equad to -7.42188
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_equad to -8.0764
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_equad to -6.62323
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_log10_equad to -7.33552
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GASP_log10_ecorr to -8.42225
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr1_2_GUPPI_log10_ecorr to -7.12549
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GASP_log10_ecorr to -8.15612
INFO: enterprise.signals.signal_base: Setting J1909-3744_Rcvr_800_GUPPI_lo

INFO: enterprise.signals.signal_base: Setting J2043+1711_L-wide_PUPPI_log10_ecorr to -7.60065
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_efac to 0.974005
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_efac to 1.09808
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_efac to 1.40923
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_efac to 1.13029
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_equad to -5.69239
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_equad to -6.3085
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GASP_log10_equad to -6.5184
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr_800_GUPPI_log10_equad to -6.28322
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GASP_log10_ecorr to -6.10416
INFO: enterprise.signals.signal_base: Setting J2145-0750_Rcvr1_2_GUPPI_log10_ecorr to

In [16]:
#Pick random initial sampling
xs = {par.name: par.sample() for par in pta.params}

# dimension of parameter space
ndim = len(xs)

# initial jump covariance matrix
cov = np.diag(np.ones(ndim) * 0.01**2)

groups = model_utils.get_parameter_groups(pta)
groups.append([ndim-2,ndim-1])

# intialize sampler
sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov, groups=groups, outDir = outdir,resume=False)

In [102]:
snames = np.unique([[qq.signal_id for qq in pp._signals] for pp in pta._signalcollections])
print(snames)

['ecorr_sherman-morrison' 'efac' 'equad' 'gw' 'linear_timing_model'
 'other_gw' 'red_noise']


Jump proposal from Jeff

In [20]:
class my_jump_proposal(model_utils.JumpProposal):
    def __init__(self,pta):
        super().__init__(pta)

    def draw_from_gw_prior(self, x, iter, beta):

        q = x.copy()
        lqxy = 0

        param_name = 'zlog10_A_gw'

        # draw parameter from signal model
        print(self.pnames)
        param = self.pnames[param_name]
        print('Here: ' + param)
        if param.size:
            idx2 = np.random.randint(0, param.size)
            q[self.pmap[str(param)]][idx2] = param.sample()[idx2]

        # scalar parameter
        else:
            q[self.pmap[str(param)]] = param.sample()

        # forward-backward jump probability
        lqxy = (param.get_logpdf(x[self.pmap[str(param)]]) -
                param.get_logpdf(q[self.pmap[str(param)]]))

        return q, float(lqxy)
    def draw_from_other_gw_prior(self, x, iter, beta):

        q = x.copy()
        lqxy = 0

        param_name = 'zlog10_A_other_gw'

        # draw parameter from signal model
        param = self.pnames[param_name]
        if param.size:
            idx2 = np.random.randint(0, param.size)
            q[self.pmap[str(param)]][idx2] = param.sample()[idx2]

        # scalar parameter
        else:
            q[self.pmap[str(param)]] = param.sample()

        # forward-backward jump probability
        lqxy = (param.get_logpdf(x[self.pmap[str(param)]]) -
                param.get_logpdf(q[self.pmap[str(param)]]))

        return q, float(lqxy)

In [21]:
#define your JP here
jp = model_utils.JumpProposal(pta)
#jp = my_jump_proposal(pta)

sampler.addProposalToCycle(jp.draw_from_prior, 15)
sampler.addProposalToCycle(jp.draw_from_red_prior, 15)
#sampler.addProposalToCycle(jp.draw_from_gw_prior, 15)
#sampler.addProposalToCycle(jp.draw_from_other_gw_prior, 15)

In [22]:
# sampler for N steps
N = int(1e5)
x0 = np.hstack(p.sample() for p in pta.params)
sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50)

/home/akaiser/.conda/envs/enterprise_dev36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


['B1855+09_red_noise_gamma', 'B1855+09_red_noise_log10_A', 'B1937+21_red_noise_gamma', 'B1937+21_red_noise_log10_A', 'B1953+29_red_noise_gamma', 'B1953+29_red_noise_log10_A', 'J0023+0923_red_noise_gamma', 'J0023+0923_red_noise_log10_A', 'J0030+0451_red_noise_gamma', 'J0030+0451_red_noise_log10_A', 'J0340+4130_red_noise_gamma', 'J0340+4130_red_noise_log10_A', 'J0613-0200_red_noise_gamma', 'J0613-0200_red_noise_log10_A', 'J0645+5158_red_noise_gamma', 'J0645+5158_red_noise_log10_A', 'J1012+5307_red_noise_gamma', 'J1012+5307_red_noise_log10_A', 'J1024-0719_red_noise_gamma', 'J1024-0719_red_noise_log10_A', 'J1455-3330_red_noise_gamma', 'J1455-3330_red_noise_log10_A', 'J1600-3053_red_noise_gamma', 'J1600-3053_red_noise_log10_A', 'J1614-2230_red_noise_gamma', 'J1614-2230_red_noise_log10_A', 'J1640+2224_red_noise_gamma', 'J1640+2224_red_noise_log10_A', 'J1643-1224_red_noise_gamma', 'J1643-1224_red_noise_log10_A', 'J1713+0747_red_noise_gamma', 'J1713+0747_red_noise_log10_A', 'J1738+0333_red_noi

TypeError: list indices must be integers or slices, not str